# 9 もっと入出力

おもしろいことの一覧

- ファイルの入出力
- 乱数生成
- コマンドライン引数

## ファイルとストリーム

ファイル読み書きの前に、ストリームデータの扱いを学ぶ。

### 入力のリダイレクト

ファイルをリダイレクトによって入力することができる。

In [1]:
import Control.Monad
import Data.Char

main = forever $ do
    l <- getLine
    putStrLn $ map toUpper l

このようなプログラムをコンパイルすると、
```sh
$ ./capsLocker < haiku.txt
```
のようにできる。

また、
```sh
$ cat haiku.txt | ./capsLocker
```
のようにもできる。

### 入力ストリームから文字列を得る

`getContents`は入力ストリームを通常の文字列にしてくれるIOアクションである。型は`getContents :: IO String`。getLineと何が違うのか。

これは入力内容を一度全て読んでメモリに読み込むのではなく、必要になるまで遅延する。

これを使うと、`forever`と`getLine`で一行ずつ処理する必要はない。

In [ ]:
import Data.Char

main = do
    contents <- getContents
    putStr $ map toUpper contents

main

HOGEHOGE
FUGA
PIY

文字列もリストなので処理は遅延されるし、`getContents`も遅延される。

`getContents`の結果が束縛されると、それは通常の文字列になるのではなく、_文字列として評価されるプロミス_になる。

それに`toUpper`が`map`されると、_関数をmapするというプロミス_になる。

最終的にputStrが呼び出されると、遅延されていた処理が実際に実行される。

----

次に、入力から10文字より短い行のみを出力するプログラムを作ってみる。

In [ ]:
main = do
    contents <- getContents -- "short\nlooooooooooooooong\nbort"
    putStr (shortLinesOnly contents)

shortLinesOnly :: String -> String
shortLinesOnly = unlines . filter (\line -> length line < 10) . lines

main

これは、`lines`によって一行分の文字列のリストになり、それが長さが10以下なら`True`を返す`lambda`によってfilterされ、`unlines`によって結合され出力される。

### 入力を変換する

`interact`関数は、「入力を文字列として受け取り、関数で変換し、結果を出力する」という一連のパターンを簡単にするためのものである。

`String -> String`型の関数を受け取り、入力にそれを適用し、結果を出力するようなIOアクションを返す。
```haskell
interact :: String -> String -> IO ()
```

これを使って、回文かどうかを判定するプログラムを書く。文字列が回文なら「回文」に、そうでないものは「回文でない」に変換される。

In [ ]:
respondPalindromes :: String -> String
respondPalindromes = unlines . map(\xs -> if isPal xs then "palindrome" else "not a palindrome") . lines

isPal :: String -> Bool
isPal xs = xs == reverse xs

main = interact respondPalindromes

main

palindrome
not a palindrome

## 9.2 ファイルの読み書き

「端末から読む」というのは、「特殊なファイルであるところの`stdin`から読んでいる」ということである。

ファイルを読むのにはSystem.IOを用いる。

In [5]:
import System.IO

main = do
    handle <- openFile "hoge.txt" ReadMode
    contents <- hGetContents handle
    putStr contents
    hClose handle

#### 1行目
`openFile`は、ファイルパスとIOModeを受け取り、ファイルに関連付けられたハンドルを返すようなIOアクションを返す。
```haskell
openFile :: FilePath -> IOMode -> IO Handle
```
`FilePath`は`String`のシノニムである。
`IOMode`は以下の用に定義されている。

```haskell
data IOMode = ReadMode | WriteMode | AppendMode | ReadWriteMode
```
これは`enum`型である。`IOMode`は`IO Mode`でないことに注意したい。後者はIOアクションになってしまう。

このアクションの結果を何かに束縛すると、そのファイルに対するハンドルを得られる。

#### 2行目
`hGetContents`は、ファイルのことを知っている`Handle`を受け取り、そのファイルに含まれる内容を返す`IO String`を返す。

つまり、`getContents`のファイル版である。なので評価は遅延する。

#### 3行目
出力している。

#### 4行目
ハンドルを受け取り、その先のファイルを閉じる`hClose`を適用している。

ハンドルが閉じられていないファイルを開こうとすると、強制終了する(ほんとに？)。

### withFile関数
Haskellでは手動でファイルを閉じないといけないのだろうか。

```haskell
withFile :: FilePath -> IOMode -> (Handle -> IO a) -> IO a
```
ファイルのパスとIOMode、それから「ハンドルを受け取ってIOアクションを返す関数」を引数に取り、「ファイルを開いて何かしてから閉じる」というIOアクションを返す。

ファイルの操作中にerrorが生じると__例外__が投げられてプログラムは停止してしまうが、`withFile`はこのようなときでも問題なくファイルを閉じてくれる。

先ほどと同じプログラムをこちらで実装してみる。

In [6]:
import System.IO

main = withFile "baabaa.txt" ReadMode $ \handle -> do
        contents <- hGetContents handle
        putStr contents

main

hogehoge

### ブラケットの時間
`withFile`のような「リソースを獲得し、それに対して何かを行い、確実にリソースを開放する」というものは便利なので、`Control.Exception`モジュールに`bracket`というものがある。
```haskell
bracket :: IO a -> (a -> IO b) -> (a -> IO c) -> IO c
```
これは以下を受け取る。
1. リソースの獲得を行うようなIOアクション
2. リソースを開放する関数
3. リソースを受け取って何事かを行う関数

`withFile`はこれを使って実装することができる。
```haskell
withFile :: FilePath -> IOMode -> (Handle -> IO a) -> IO a
withFile name mode f = bracket (openFile name mode) (\handle -> hClose handle) (\handle -> f handle) 
```

### ハンドルを握れ！
`hGetContents`と`getContents`のような関係が、以下のものにも存在する。
* hGetLine <==> getLine
* hPutStr <==> putStr
* hPutStrLn <==> putStrLn
* hGetChar <==> getChar

ファイルを読み込んで文字列として扱うのは非常に一般的な操作なので、関連した関数が3つ用意されている。

#### readFile
```haskell
readFile :: FilePath -> IO String
```
`readFile`はファイルパスを受け取り、ファイルの内容を表す文字列を返すIOアクションを返す（もちろん遅延する）。

In [7]:
import System.IO

main = do
    contents <- readFile "baabaa.txt"
    putStr contents

main

hogehoge

readFileは自動でハンドルを閉じる。
#### writeFile
```haskell
writeFile :: FilePath -> String -> IO ()
```
ファイルパス、書き込む文字列を受け取り、出力するIOアクションを返す。ファイルは上書きされる。

#### appendFile
上記の追記版。

## 9.3 ToDoリスト

TODOリストを作る　創刊号は事務書類がついて980円

In [8]:
import System.IO

main = do
    todoItem <- getLine
    appendFile "todo.txt" (todoItem ++ "\n")

### アイテムの削除

健常な人間はTODOを消化するので削除ができる必要がある。そのために`System.Directory`の関数を使う。

In [9]:
import System.IO
import System.Directory
import Data.List

main = do
    contents <- readFile "todo.txt"
    let todoTasks = lines contents -- 行毎に分割
        numberedTasks = zipWith(\n line -> show n ++ " - " ++ line) [0..] todoTasks -- 番号を振ったタスクリストを作成

    putStrLn "there are your TO-DO items: "
    mapM_ putStrLn numberedTasks

    putStrLn "Which do you want to delete?"
    numberString <- getLine -- 消去する番号を読み取る
    let number = read numberString -- 数字として読み込む
        newTodoItems = unlines $ delete (todoTasks !! number) todoTasks -- todoTasksからその番号のタスクを削除

    (tempName, tempHandle) <- openTempFile "." "temp" -- tempなる名前のファイルをカレントディレクトリに作成
    hPutStr tempHandle newTodoItems
    hClose tempHandle
    removeFile "todo.txt"
    renameFile tempName "todo.txt"

概ねよいが、これでは一時ファイルを開いているときにプログラムが異常終了すると、一時ファイルが削除されずに残ってしまう。

### クリーンアップ

`bracketOnError`という関数を使う。`bracket`と違って、これは`error`が生じた時にのみ作動する。

In [10]:
import System.IO
import System.Directory
import Data.List
import Control.Exception

main = do
    contents <- readFile "todo.txt"
    let todoTasks = lines contents
        numberedTasks = zipWith(\n line -> show n ++ " - " ++ line) [0..] todoTasks 
    putStrLn "there are your TO-DO items: "
    mapM_ putStrLn numberedTasks
    putStrLn "Which do you want to delete?"
    numberString <- getLine
    let number = read numberString
        newTodoItems = unlines $ delete (todoTasks !! number) todoTasks

-- ここから異なる

    bracketOnError (openTempFile "." "temp")
        (\(tempName, tempHandle) -> do -- エラー処理
            hClose tempHandle           -- tempを閉じて
            removeFile tempName)        -- 消去する #575
        (\(tempName, tempHandle) -> do -- 通常時は上と同じ
            hPutStr tempHandle newTodoItems
            hClose tempHandle
            removeFile "todo.txt"
            renameFile tempName "todo.txt")

### 9.4 コマンドライン引数
`System.Environment`にある、`getArgs`と`getProgName`を使う。
```haskell
getArgs :: IO [String]
getProgName :: IO String
```
これらは名前と型の通りの挙動をする。

In [1]:
import System.Environment
import Data.List

main = do
    args <- getArgs -- argv[1], argv[2], ...
    progName <- getProgName -- argv[0]
    putStrLn "the arguments are: "
    mapM_ putStrLn args
    putStrLn "the program name is: "
    putStrLn progName
    
main

the arguments are: 
kernel
/root/.local/share/jupyter/runtime/kernel-b533b1ff-d555-4e78-882b-7c72a2b38775.json
--ghclib
/root/.stack/programs/x86_64-linux/ghc-7.10.2/lib/ghc-7.10.2
the program name is: 
ihaskell

## 9.5 ToDoリストをもっと楽しむ

要求仕様
* タスクを閲覧する
    * `./todo view todo.txt`
* タスクを追加する
    * `./todo add todo.txt "find the magic sword of power"`
* タスクを削除する
    * `./todo remove todo.txt 2`
    
### マルチタスクタスクリスト

まずは、コマンドを受けとり、引数リストを受け取って動作を行うIOアクションを返す関数を作る。高階関数が便利過ぎる。

In [12]:
import System.Environment
import System.Directory
import System.IO
import Data.List

dispatch :: String -> [String] -> IO ()
dispatch "add" = add
dispatch "view" = view
dispatch "remove" = remove

main = do
    (command:argList) <- getArgs -- 最初の引数はcommandに格納
    dispatch command argList

add :: [String] -> IO () -- ここでは不正な入力は考慮しない
add [fileName, todoItem] = appendFile fileName (todoItem ++ "\n")

view :: [String] -> IO () -- removeするときと基本的に同じ
view [fileName] = do 
    contents <- readFile fileName
    let todoTasks = lines contents
        numberedTasks = zipWith(\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStr $ unlines numberedTasks

remove :: [String] -> IO ()
remove [fileName, numberString] = do -- さっき実装したものと同じ
    contents <- readFile fileName
    let todoTasks = lines contents
        numberedTasks = zipWith(\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStrLn "there are your TO-DO items: "
    mapM_ putStrLn numberedTasks
    putStrLn "Which do you want to delete?"
    numberString <- getLine
    let number = read numberString
        newTodoItems = unlines $ delete (todoTasks !! number) todoTasks
    bracketOnError (openTempFile "." "temp")
        (\(tempName, tempHandle) -> do
            hClose tempHandle
            removeFile tempName)
        (\(tempName, tempHandle) -> do
            hPutStr tempHandle newTodoItems
            hClose tempHandle
            removeFile "todo.txt"
            renameFile tempName "todo.txt")

### 不正な入力に対応する
dispatch関数の最後に、全てを拾うパターンを追加し、存在しないコマンドにはそれなりのメッセージを返す。

In [13]:
import System.Environment
import System.Directory
import System.IO
import Data.List

dispatch :: String -> [String] -> IO ()
dispatch "add" = add
dispatch "view" = view
dispatch "remove" = remove
dispatch command = doesntExist command

doesntExist :: String -> [String] -> IO ()
doesntExist command _ = putStrLn $ "The " ++ command ++ " command doesn't exist"

さらに、addなどの引数がおかしかったとき用のパターンも用意しておく。パターンマッチが便利だ。

In [14]:
add :: [String] -> IO ()
add [fileName, todoItem] = appendFile fileName (todoItem ++ "\n")
add _ = putStrLn "The add command takes exactly two arguments"

まだmainで引数リストが一つ以上だとしていること、またファイルが存在するかどうかを確認していないことなどが残っている。

## 9.6 ランダム性

Haskellでは参照透明性のおかげで乱数生成ルーチンが面倒になってしまう

`System.Random`モジュールの`random`は、
```haskell
random :: (RandomGen g, Random a) => g -> (a, g)
```
のようになっている。`RandomGen`型クラスはランダム性を生成するものの型、`Random`型クラスはランダムな値になり得る型である。

つまり、これは乱数生成器を受け取り、乱数と（アップデートされた）乱数生成器を返す。

`mkStdGen`を使うことで、乱数生成機を作ることができる。

In [15]:
import System.Random

random (mkStdGen 100) :: (Int, StdGen)
random (mkStdGen 100) :: (Int, StdGen) -- 同じ呼び出しなので同じものが来るはず
random (mkStdGen 999) :: (Int, StdGen) -- 違うはず

(-3633736515773289454,693699796 2103410263)

(-3633736515773289454,693699796 2103410263)

(-6423851212088117412,2063054562 2103410263)

### コイントス
連続で乱数を出すには、前回生成された乱数生成機を使えばよい。

In [16]:
import System.Random

threeCoins :: StdGen -> (Bool, Bool, Bool)
threeCoins gen = 
    let (firstCoin, newGen) = random gen :: (Bool, StdGen) -- 抜かすと怒られてしまった
        (secondCoin, newGen') = random newGen :: (Bool, StdGen)
        (thirdCoin, newGen'') = random newGen' :: (Bool, StdGen)
    in (firstCoin, secondCoin, thirdCoin)

threeCoins (mkStdGen 1234)
threeCoins (mkStdGen 567)
threeCoins (mkStdGen 89)

(True,True,True)

(True,False,False)

(True,False,False)

### ランダムな関数をもっと

`randoms`はジェネレータを受け取って乱数の入った無限リストを返す

In [17]:
import System.Random

take 5 $ randoms (mkStdGen 11) :: [Int]

[5260538044923710387,4361398698747678847,-8221315287270277529,7278185606566790575,1652507602255180489]

`randoms`は`([a], g)`のような形では生成器は返せない。無限の長さのリストを返すからだ。

In [18]:
import System.Random

randoms' :: (RandomGen g, Random a) => g -> [a]
randoms' gen =
    let (value, newGen) = random gen
    in value:randoms' newGen

take 5 $ randoms' (mkStdGen 11) :: [Int]

No instance for (Random t0)
The type variable `t0' is ambiguous
When checking that `newGen' has the inferred type
  newGen :: g
Probable cause: the inferred type is ambiguous
In the expression: let (value, newGen) = random gen in value : interactive:Ghci330.randoms' newGen
In an equation for randoms': randoms' gen = let (value, newGen) = random gen in value : interactive:Ghci330.randoms' newGen

Could not deduce (RandomGen g0) arising from a use of interactive:Ghci330.randoms'
from the context (RandomGen g, Random a) bound by the type signature for interactive:Ghci330.randoms' :: (RandomGen g, Random a) => g -> [a] at <interactive>:1:13-47
The type variable `g0' is ambiguous
Note: there is a potential instance available: instance RandomGen StdGen -- Defined in `System.Random'
In the second argument of `(:)', namely `interactive:Ghci330.randoms' newGen'
In the expression: value : interactive:Ghci330.randoms' newGen
In the expression: let (value, newGen) = random gen in value : interactive:Ghci330.randoms' newGen

型推論に失敗しておられるので困る

In [19]:
import System.Random

randoms' :: (RandomGen g, Random a) => g -> [(a, g)]
randoms' gen =
    let (value, newGen) = random gen
    in (value, newGen):randoms' newGen

No instance for (Random t0)
The type variable `t0' is ambiguous
When checking that `newGen' has the inferred type
  newGen :: g
Probable cause: the inferred type is ambiguous
In the expression: let (value, newGen) = random gen in (value, newGen) : interactive:Ghci330.randoms' newGen
In an equation for randoms': randoms' gen = let (value, newGen) = random gen in (value, newGen) : interactive:Ghci330.randoms' newGen

毎ステップでnewGenも一緒になら返せるっぽい

返すリストが有限でよいなら、ジェネレータも返すことができる。

In [20]:
import System.Random

finiteRandoms' :: (RandomGen g, Random a, Num n) => n -> g -> ([a], g)
finiteRandoms' 0 gen = ([], gen)
finiteRandoms' n gen =
    let (value, newGen) = random gen
        (restOfList, finalGen) = finiteRandoms' (n-1) newGen
    in (value:restOfList, finalGen)

No instance for (Random t0)
The type variable `t0' is ambiguous
When checking that `newGen' has the inferred type
  newGen :: g
Probable cause: the inferred type is ambiguous
In the expression:
  let
    (value, newGen) = random gen
    (restOfList, finalGen) = interactive:Ghci330.finiteRandoms' (n - 1) newGen
  in (value : restOfList, finalGen)
In an equation for finiteRandoms':
    finiteRandoms' n gen
      = let
          (value, newGen) = random gen
          (restOfList, finalGen) = interactive:Ghci330.finiteRandoms' (n - 1) newGen
        in (value : restOfList, finalGen)

Could not deduce (RandomGen t0)
from the context (Random a) bound by the inferred type for `restOfList': Random a => [a] at <interactive>:5:9-60
The type variable `t0' is ambiguous
When checking that `restOfList' has the inferred type
  restOfList :: forall a. Random a => [a]
Probable cause: the inferred type is ambiguous
In the expression:
  let
    (value, newGen) = random gen
    (restOfList, finalGen) = interactive:Ghci330.finiteRandoms' (n - 1) newGen
  in (value : restOfList, finalGen)
In an equation for finiteRandoms':
    finiteRandoms' n gen
      = let
          (value, newGen) = random gen
          (restOfList, finalGen) = interactive:Ghci330.finiteRandoms' (n - 1) newGen
        in (value : restOfList, finalGen)

Could not deduce (Random a0)
from the context (RandomGen t) bound by the inferred type for `finalGen': RandomGen t => t at <interactive>:5:9-60
The type variable `a0' is ambiguous
When checking that `finalGen' has the inferred type
  finalGen :: forall t. RandomGen t => t
Probable cause: the inferred type is ambiguous
In the expression:
  let
    (value, newGen) = random gen
    (restOfList, finalGen) = interactive:Ghci330.finiteRandoms' (n - 1) newGen
  in (value : restOfList, finalGen)
In an equation for finiteRandoms':
    finiteRandoms' n gen
      = let
          (value, newGen) = random gen
          (restOfList, finalGen) = interactive:Ghci330.finiteRandoms' (n - 1) newGen
        in (value : restOfList, finalGen)

Could not deduce (Eq n) arising from the literal `0'
from the context (RandomGen g, Random a, Num n) bound by the type signature for interactive:Ghci330.finiteRandoms' :: (RandomGen g, Random a, Num n) => n -> g -> ([a], g) at <interactive>:1:19-70
Possible fix: add (Eq n) to the context of the type signature for interactive:Ghci330.finiteRandoms' :: (RandomGen g, Random a, Num n) => n -> g -> ([a], g)
In the pattern: 0
In an equation for finiteRandoms': finiteRandoms' 0 gen = ([], gen)

生成される乱数の範囲を指定したい場合は、`randomR`を用いる。
```haskell
randomR :: (RandomGen g, Random a) => (a, a) -> g -> (a, g)
```
のようになっている。これは上限と下限のペアを受け取って、その範囲内の乱数を返す。

これの無限リスト版として、`randomRs`が用意されている。

In [21]:
import System.Random

randomR (1,6) (mkStdGen 3445234)
take 10 $ randomRs ('a', 'z') (mkStdGen 3) :: String

(2,418685258 40692)

"xnuhlfwywq"

### ランダム性とIO

`System.Random`には、`getStdGen`なる`IO StdGen`型のIOアクションがある。これによってシステムのグローバル乱数生成器を初期化できる。

In [1]:
import System.Random

main = do
    gen <- getStdGen
    putStrLn $ take 20 $ randomRs ('a', 'z') gen
    gen2 <- getStdGen
    putStrLn $ take 20 $ randomRs ('a', 'z') gen2 -- 何度やっても同じ！

main

enrxcavkhmwqufzsjmln
enrxcavkhmwqufzsjmln

二つの異なる文字列を得るには、`newStdGen`を用いる。これによって新しい乱数生成器が作られ、またグローバル生成器も更新される。

In [6]:
import System.Random

main = do
    gen <- getStdGen
    putStrLn $ take 20 $ randomRs ('a', 'z') gen
    gen2 <- newStdGen
    putStrLn $ take 20 $ randomRs ('a', 'z') gen2
    
main

sxehgiazlshuctimxjaf
usfghwmcnfgxuuhukrgp

これを使って数当てゲームができるようになる。

In [24]:
import System.Random
import Control.Monad(when)

main = do
    gen <- getStdGen
    askForNumber gen
    
askForNumber :: StdGen -> IO ()
askForNumber gen = do
    let (randNumber, newGen) = randomR (1,10) gen :: (Int, StdGen) -- 乱数を思い描く
    putStrLn "Which number in the range [1,10] am I thinking of?"
    numberString <-getLine
    when (not $ null numberString) $ do -- 入力が空でないか確認
        let number = read numberString
        if randNumber == number
            then putStrLn "you are correct!"
            else putStrLn $ "Sorry, it was " ++ show randNumber
        askForNumber newGen

## 9.7 bytestring
リストは非常に便利だが、ファイルを文字列にするときはランタイムコストに注意しなければならない。

リストは遅延評価されるので、例えば`[1,2,3,4]`は`1:2:3:4:[]`と等価だが、その先頭要素を処理しても残りの`2:3:4:[]`は未評価である。これを__サンク__と呼ぶ。Haskellはサンクによって計算を遅延している。

リストは、必要になると先頭の要素と後続要素になるはずのプロミスを渡してくれるようなプロミスであると考えられる。

大きなファイルを読み込んだりすると、これによるオーバーヘッドが問題になることが多い。それを解決するために、`bytestring`が存在する。

`bytestring`はリストに似たデータ構造であるが、よそは1バイトに固定されており、遅延評価を行うための方法も異なる。

### 正格bytestringと遅延bytestring

正格bytestringは`Data.ByteString`で提供されており、遅延評価を行わない。

遅延bytestringは`Data.ByteString.Lazy`で提供されており、遅延評価を行うが、64キロバイト（L2キャッシュを念頭に置いて作られている）のチャンク単位で格納されており、遅延評価もこの単位で行われる。これによって、要素数と同じ量のサンクがあるリストよりも高速に処理することができる。

`Data.ByteString.Lazy`では`Data.List`と同様の関数が定義されている。

#### pack
```haskell
pack :: [Word8] -> ByteString
```
は、Word8型のリストをByteStringに格納する。ここで、`Word8`は8bit符号なし整数であり、Num型クラスのインスタンスである。

数からbytestringにpackし、またunpackするには以下のようにする。

In [25]:
import Data.ByteString.Lazy

let bys = pack [99, 97, 110]
unpack bys

[99,97,110]

packする値が少なければ、一つのチャンクに収まる。また、数が`Word8`であることは強制されるので、明示する必要はない。
もし大きい数を入れると、オーバーフローする。

正格bytestringと遅延bytestringは相互に変換できる。

In [26]:
import qualified Data.ByteString.Lazy as B
import qualified Data.ByteString as S

B.fromChunks [S.pack [40,41,42], S.pack [43, 44, 45], S.pack [46, 47, 48]]

"()*+,-./0"

bytestringでは`:`は`cons`である。

それ以外の関数は、`Data.List`と同じインターフェースになっている。

### bytestringを使ったファイルのコピー
`cp`コマンドを作る。`System.Directory`の`copyFile`は使わない。

In [27]:
import System.Environment
import System.Directory
import System.IO
import Control.Exception
import qualified Data.ByteString.Lazy as B

main = do
    (fileName1:fileName2:_) <- getArgs
    copy fileName1 fileName2

copy source dest = do
    contents <- B.readFile source -- B.readFileによってファイルを読む
    bracketOnError
        (openTempFile "." "temp") -- 一時ファイルをカレントディレクトリで開く
        (\(tempName, tempHandle) -> do -- 何かが起きたときにクリーンアップするためのラムダ
            hClose tempHandle
            removeFile tempName)
        (\(tempName, tempHandle) -> do
            B.hPutStr tempHandle contents -- B.hPutStrによって一時ファイルに内容を書き込む
            hClose tempHandle
            renameFile tempName dest)

bytestring用の関数が必要なのは読み書きの時だけ